<a href="https://colab.research.google.com/github/Yetibi/Autoreel/blob/main/LIMPIEZA_Y_PROCESAMIENTO_ARCHIVOS_MENSUALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 1. MONTAR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

# 2. IMPORTAR LIBRERÍAS
import pandas as pd
import numpy as np

# 3. DEFINIR RUTA DEL ARCHIVO UNIFICADO
ruta_archivo = "/content/drive/MyDrive/AUTOREEL CONSULTORIA/AUTOREEL_CONSOLIDADO_MAYO_2024.xlsx"
nombre_hoja = "UNIFICADO"

# 4. CARGAR ARCHIVO Y HOJA
df = pd.read_excel(ruta_archivo, sheet_name=nombre_hoja)

# 5. LLAMAR A LA FUNCIÓN DE LIMPIEZA
def procesar_mes(df):
    df.columns = df.columns.str.strip()

    # --- Renombrar columnas clave ---
    df = df.rename(columns={
        "SERVICIO": "CONCEPTO",
        "TOTAL PAGADO": "VALOR ($)",
        "BANCO": "BANCO VALOR PAGOS TRANSFERENCIAS",
        "ORDEN": "BANCO INGRESO EFECTIVO",
        "TRANSFERENCIA": "INGRESO POR TRANSFERENCIA"
    })

    # --- Eliminar encabezados repetidos en SERVICIOS ---
    encabezados_servicio = ["# ORDEN", "CONCEPTO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
                            "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "VALOR ($)"]

    def es_fila_encabezado(fila):
        if str(fila.get("CATEGORIA", "")).strip().upper() != "SERVICIOS":
            return False
        coincidencias = sum(
            str(fila.get(col, "")).strip().upper() == col.strip().upper()
            for col in encabezados_servicio
        )
        return coincidencias >= len(encabezados_servicio) - 1

    df = df[~df.apply(es_fila_encabezado, axis=1)].reset_index(drop=True)

    # --- Clasificar tipo de movimiento ---
    df["CATEGORIA"] = df["CATEGORIA"].astype(str).str.strip().str.upper()
    df["TIPO MOVIMIENTO"] = df.apply(
        lambda row: "EGRESO" if row["CATEGORIA"] in [
            "ENTREGA EFECTIVO DIA A", "ADELANTOS O PRESTAMOS", "GASTOS Ò PAGOS PROVEEDORES",
            "TRANSFERENCIAS", "VALOR PAGOS TRANSFERENCIAS", "COMISION"
        ] else "INGRESO" if row["CATEGORIA"] in ["BASE DÍA", "INGRESO EFECTIVO", "SERVICIOS"]
        else row.get("TIPO MOVIMIENTO", None),
        axis=1
    )

    # --- Asignar FORMA DE PAGO ---
    def detectar_forma_pago(row):
        formas = []
        if isinstance(row.get("EFECTIVO"), (int, float)) and row["EFECTIVO"] > 0:
            formas.append("EFECTIVO")
        if isinstance(row.get("BANCOLOMBIA"), (int, float)) and row["BANCOLOMBIA"] > 0:
            formas.append("BANCOLOMBIA")
        if isinstance(row.get("DATAFONO"), (int, float)) and row["DATAFONO"] > 0:
            formas.append("DATAFONO")
        return "/".join(formas) if formas else None

    df.insert(df.columns.get_loc("BANCOLOMBIA"), "FORMA DE PAGO", df.apply(detectar_forma_pago, axis=1))

    # --- Reemplazos especiales en CONCEPTO ---
    df["CONCEPTO"] = df["CONCEPTO"].replace({
        "Julian": "Comisión Administrativa",
        "Esteban": "Comisión Operativa"
    })
    df = df[~df["CONCEPTO"].isin(["COMISIÓN ADMINISTRATIVA", "COMISIÓN OPERATIVA"])]

    # --- Reubicar PERSONA / GASTOS / TRANSFERENCIAS / BASE DÍA ---
    df.loc[df["PERSONA"].notna(), "CONCEPTO"] = df.loc[df["PERSONA"].notna(), "PERSONA"]
    df.loc[df["PERSONA"].notna(), "VALOR ($)"] = df.loc[df["PERSONA"].notna(), "VALOR"]
    df.loc[df["CATEGORIA"] == "GASTOS Ò PAGOS PROVEEDORES", ["CONCEPTO", "VALOR ($)"]] = df.loc[
        df["CATEGORIA"] == "GASTOS Ò PAGOS PROVEEDORES", ["GASTOS Ò PAGOS PROVEEDORES", "VALOR"]
    ].values
    df.loc[df["CATEGORIA"] == "BASE DÍA", "VALOR ($)"] = df.loc[df["CATEGORIA"] == "BASE DÍA", "BASE DÍA"]
    df.loc[df["CATEGORIA"] == "BASE DÍA", "CONCEPTO"] = "Base día"

    # --- Eliminar filas con valores no válidos ---
    def es_valido(x):
        try:
            return float(x) > 0
        except:
            return False

    df = df[~((df["CATEGORIA"] == "ADELANTOS O PRESTAMOS") & ~df["VALOR ($)"].apply(es_valido))]
    df = df[~((df["CATEGORIA"] == "ENTREGA EFECTIVO DIA A") & ~df["VALOR ($)"].apply(es_valido))]
    df = df[~((df["CATEGORIA"] == "INGRESO EFECTIVO") & (~df["VALOR ($)"].apply(es_valido)))].reset_index(drop=True)

    # --- Reubicar datos de INGRESO EFECTIVO a CONCEPTO y VALOR ($) ---
    df.loc[df["CATEGORIA"] == "INGRESO EFECTIVO", "CONCEPTO"] = df.loc[df["CATEGORIA"] == "INGRESO EFECTIVO", "INGRESO EFECTIVO"]

    # --- Reglas EFECTIVO INGRESO / EGRESO ---
    def asignar_efectivos(row):
        cat = row["CATEGORIA"]
        val = row["VALOR ($)"]
        efectivo = row["EFECTIVO"]
        if cat in ["BASE DÍA", "INGRESO EFECTIVO"]:
            return pd.Series([val, 0])
        elif cat == "SERVICIOS" and isinstance(efectivo, (int, float)) and efectivo > 0:
            return pd.Series([efectivo, 0])
        elif cat in ["ADELANTOS O PRESTAMOS", "ENTREGA EFECTIVO DIA A", "GASTOS Ò PAGOS PROVEEDORES"]:
            return pd.Series([0, val])
        else:
            return pd.Series([None, None])

    df[["EFECTIVO INGRESO", "EFECTIVO EGRESO"]] = df.apply(asignar_efectivos, axis=1)

    # --- EGRESO POR TRANSFERENCIA ---
    df["EGRESO POR TRANSFERENCIA"] = df["CATEGORIA"] == "TRANSFERENCIAS"
    df["EGRESO POR TRANSFERENCIA"] = df["EGRESO POR TRANSFERENCIA"].map({True: "Si", False: "No"})

    # --- Orden final ---
    columnas_final = [
        "FECHA", "# ORDEN", "CONCEPTO", "CATEGORIA", "TIPO MOVIMIENTO", "CLIENTE", "CELULAR",
        "VEHICULO", "PLACA", "OPERARIO", "FORMA DE PAGO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO",
        "INGRESO POR TRANSFERENCIA", "VALOR ($)", "BANCO VALOR PAGOS TRANSFERENCIAS", "BANCO INGRESO EFECTIVO",
        "EFECTIVO INGRESO", "EFECTIVO EGRESO", "EGRESO POR TRANSFERENCIA"
    ]
    df = df[[col for col in columnas_final if col in df.columns]]

    return df

# 6. PROCESAR
df_tratada = procesar_mes(df)
df_tratada.head()

from openpyxl import load_workbook

# Ruta al archivo existente
ruta_archivo = "/content/drive/MyDrive/AUTOREEL CONSULTORIA/AUTOREEL_CONSOLIDADO_MAYO_2024.xlsx"

# Abrimos el archivo existente sin borrarlo
with pd.ExcelWriter(ruta_archivo, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_tratada.to_excel(writer, sheet_name="tabla transformada", index=False)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AUTOREEL CONSULTORIA/AUTOREEL_CONSOLIDADO_MAYO_2024.xlsx'

In [10]:
import pandas as pd
import numpy as np

# Paso 1: Cargar el DataFrame inicial
df = xl("'UNIFICADA ORIGINAL'!A1:W2578", headers=True)

# Paso 2: Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Paso 3: Lista de columnas clave
columnas_clave = [
    "SERVICIO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
    "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "TOTAL PAGADO"
]

# Paso 4: Limpiar espacios internos en columnas clave
df[columnas_clave] = df[columnas_clave].applymap(
    lambda x: str(x).strip() if isinstance(x, str) else x
)

# Paso 5: Eliminar filas vacías en "SERVICIO"
def fila_vacia_en_servicio(fila):
    if fila["CATEGORIA"].strip().upper() != "SERVICIOS":
        return False
    return all(
        pd.isna(fila[col]) or str(fila[col]).strip() in ["", "0", "None", "nan"]
        for col in columnas_clave
    )
df_tratada = df[~df.apply(fila_vacia_en_servicio, axis=1)].reset_index(drop=True)

# Paso 6: Eliminar filas que son encabezados repetidos en "SERVICIOS"
encabezados_servicio = [
    "# ORDEN", "SERVICIO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
    "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "TOTAL PAGADO"
]

def es_fila_encabezado(fila):
    if str(fila.get("CATEGORIA", "")).strip().upper() != "SERVICIOS":
        return False
    coincidencias = sum(
        str(fila.get(col, "")).strip().upper() == col.strip().upper()
        for col in encabezados_servicio
    )
    return coincidencias >= len(encabezados_servicio) - 1

df_tratada = df_tratada[~df_tratada.apply(es_fila_encabezado, axis=1)].reset_index(drop=True)

# Paso 7: Renombrar columnas
df_tratada = df_tratada.rename(columns={
    "SERVICIO": "CONCEPTO",
    "TOTAL PAGADO": "VALOR ($)"
})

# Paso 8: Crear columna "TIPO MOVIMIENTO"
categorias_egreso = [
    "COMISION", "GASTOS Ò PAGOS PROVEEDORES", "VALOR PAGOS TRANSFERENCIAS", "ENTREGA EFECTIVO DIA A:"
]
df_tratada["TIPO MOVIMIENTO"] = df_tratada["CATEGORIA"].apply(
    lambda x: "EGRESO" if str(x).strip().upper() in categorias_egreso else "INGRESO"
)

# Paso 9: Crear columna "FORMA DE PAGO"
def detectar_forma_pago(row):
    formas = []
    if row.get("EFECTIVO", 0) > 0:
        formas.append("EFECTIVO")
    if row.get("BANCOLOMBIA", 0) > 0:
        formas.append("BANCOLOMBIA")
    if row.get("DATAFONO", 0) > 0:
        formas.append("DATAFONO")
    return "/".join(formas) if formas else None

df_tratada["FORMA DE PAGO"] = df_tratada.apply(detectar_forma_pago, axis=1)

# Paso 10: Asegurar valores comparables
df_tratada["CATEGORIA"] = df_tratada["CATEGORIA"].astype(str).str.strip().str.upper()

# Paso 11: Asignación condicional de "EFECTIVO INGRESO" y "EFECTIVO EGRESO"
def asignar_efectivos(row):
    categoria = row["CATEGORIA"]
    valor = row["VALOR ($)"]
    efectivo = row["EFECTIVO"]

    if categoria in ["BASE DÍA", "INGRESO EFECTIVO"]:
        return pd.Series([valor, 0])
    elif categoria == "SERVICIOS" and isinstance(efectivo, (int, float)) and efectivo > 0:
        return pd.Series([efectivo, 0])
    elif categoria in ["ADELANTOS O PRESTAMOS", "ENTREGA EFECTIVO DIA A", "GASTOS Ò PAGOS PROVEEDORES"]:
        return pd.Series([0, valor])
    else:
        return pd.Series([row.get("EFECTIVO", 0), None])

df_tratada[["EFECTIVO INGRESO", "EFECTIVO EGRESO"]] = df_tratada.apply(asignar_efectivos, axis=1)

# Paso 12: Clasificar movimientos y columnas finales
columnas_final = [
    "FECHA", "# ORDEN", "CONCEPTO", "CATEGORIA", "TIPO MOVIMIENTO", "CLIENTE", "CELULAR",
    "VEHICULO", "PLACA", "OPERARIO", "FORMA DE PAGO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO",
    "INGRESO POR TRANSFERENCIA", "VALOR ($)", "BANCO VALOR PAGOS TRANSFERENCIAS", "BANCO INGRESO EFECTIVO",
    "EFECTIVO INGRESO", "EFECTIVO EGRESO"
]
df_tratada = df_tratada[[col for col in columnas_final if col in df_tratada.columns]]

# Resultado final
df_tratada

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AUTOREEL CONSULTORIA/BASES DE DATOS SOURCE/AUTOREEL MAYO 2024.xlsx'